# BigData Analysis Term-Project

## AI융합학부 20160348 강민성

## 주제: 2007-2019년에 서울에서 발생한 교통사고 분석

## 1. Data Preprocessing

### 1) 수집한 데이터 셋 목록
- 교통사고분석시스템(TAAS)

    * 2014_19년 서울_경기 개별 교통사고 정보.xlsx
    * 서울시 개별사고정보(2007_2009).xlsx
    * 서울시 개별사고정보(2010_2018).xlsx
    
    
- 서울시 열린데이터 광장
    * Seoul_population.txt (2007~2019년 서울시 자치구별 인구 데이터)

In [1]:
# 데이터 전처리에 필요한 모듈 import
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

### 2) 교통사고정보 데이터 전처리

In [2]:
# Data Load
Seoul_0709 = pd.read_excel('..\data\input\서울시 개별사고정보(2007_2009).xlsx')
Seoul_1018 = pd.read_excel('..\data\input\서울시 개별사고정보(2010_2018).xlsx')
Seoul_Gyeonggi_1419 = pd.read_excel('..\data\input\서울경기 개별사고정보(2014_2019).xlsx')

In [3]:
# Column명  확인
print(Seoul_0709.columns, len(Seoul_0709.columns))
print(Seoul_1018.columns, len(Seoul_1018.columns))
print(Seoul_Gyeonggi_1419.columns, len(Seoul_Gyeonggi_1419.columns))

Index(['발생일', '발생시간', '요일', '발생지_시군구', '법정동명', '사고내용', '사망자수', '중상자수', '경상자수',
       '부상신고자수', '사고유형_대분류', '사고유형_중분류', '사고유형', '가해자법규위반', '노면상태_대분류', '노면상태',
       '기상상태', '도로형태_대분류', '도로형태', '가해자차종', '가해자성별', '가해자연령', '가해자신체상해정도',
       '피해자차종', '피해자성별', '피해자연령', '피해자신체상해정도'],
      dtype='object') 27
Index(['발생일', '발생시간', '요일', '발생지_시군구', '사고내용', '법정동명', '사망자수', '중상자수', '경상자수',
       '부상신고자수', '사고유형_대분류', '사고유형_중분류', '사고유형', '가해자법규위반', '노면상태_대분류', '노면상태',
       '기상상태', '도로형태_대분류', '도로형태', '가해자차종', '가해성별', '가해자연령', '가해자신체상해정도',
       '피해자차종', '피해자성별', '피해자연령', '피해자신체상해정도'],
      dtype='object') 27
Index(['발생일', '발생시간', '사망자수', '중상자수', '경상자수', '부상신고자수', '발생지_시도', '발생지_시군구',
       '법정동명', '사고유형_대분류', '사고유형_중분류', '사고유형', '가해자법규위반', '도로형태_대분류', '도로형태',
       '노면상태_대분류', '노면상태', '기상상태', '가해자차종', '가해자성별', '가해자연령', '가해자신체상해정도',
       '피해자차종', '피해자성별', '피해자연령', '피해자신체상해정도'],
      dtype='object') 26


In [4]:
Seoul_1018.rename(columns={'가해성별':'가해자성별'}, inplace=True)

Seoul_Gyeonggi_1419['발생일'] = Seoul_Gyeonggi_1419['발생일'].astype(str)

In [5]:
# 2019년 이전 레코드 삭제
year1418_idx = Seoul_Gyeonggi_1419[(Seoul_Gyeonggi_1419['발생일'].str.slice(start=0, stop=4) == '2014') |
                                   (Seoul_Gyeonggi_1419['발생일'].str.slice(start=0, stop=4) == '2015') |
                                   (Seoul_Gyeonggi_1419['발생일'].str.slice(start=0, stop=4) == '2016') |
                                   (Seoul_Gyeonggi_1419['발생일'].str.slice(start=0, stop=4) == '2017') |
                                   (Seoul_Gyeonggi_1419['발생일'].str.slice(start=0, stop=4) == '2018')].index
Seoul_Gyeonggi_1419.drop(year1418_idx, inplace=True)

# 경기도에서 발생한 사고 레코드 삭제
Gyeonggi_idx = Seoul_Gyeonggi_1419[Seoul_Gyeonggi_1419['발생지_시도'] == '경기'].index
Seoul_Gyeonggi_1419.drop(Gyeonggi_idx, inplace=True)
Seoul_Gyeonggi_1419.reset_index(inplace=True, drop=True) # index 재정렬

# Seoul_Gyeonggi_1419에 '요일' Column 추가
weekday_name = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'} 
Seoul_Gyeonggi_1419['요일'] = pd.to_datetime(Seoul_Gyeonggi_1419['발생일']).dt.weekday.map(weekday_name)

In [6]:
# Seoul_Gyeonggi_1419에 '사고내용' Column 추가
case_condition = [
    (Seoul_Gyeonggi_1419['사망자수'] > 0) ,
    (Seoul_Gyeonggi_1419['중상자수'] > 0),
    (Seoul_Gyeonggi_1419['경상자수'] > 0),
    (Seoul_Gyeonggi_1419['부상신고자수'] > 0)]
caselist = ['사망', '중상', '경상', '부상신고']
Seoul_Gyeonggi_1419['사고내용'] = np.select(case_condition, caselist)

# '발생시_시도' Column은 서울 데이터밖에 없으므로 삭제
Seoul_Gyeonggi_1419.drop(['발생지_시도'], axis=1, inplace=True)

# 중간결과 출력
Seoul_Gyeonggi_1419

,발생일,발생시간,사망자수,중상자수,경상자수,부상신고자수,발생지_시군구,법정동명,사고유형_대분류,사고유형_중분류,...,가해자차종,가해자성별,가해자연령,가해자신체상해정도,피해자차종,피해자성별,피해자연령,피해자신체상해정도,요일,사고내용
0,20190101,00시,0,0,1,0,강서구,방화동,차대사람,횡단중,...,승용차,남,26세,상해없음,보행자,남,40세,경상,화,경상
1,20190101,00시,0,0,1,0,구로구,고척동,차대차,추돌,...,이륜차,남,23세,상해없음,승용차,남,71세,경상,화,경상
2,20190101,00시,0,0,1,0,서초구,서초동,차대차,기타,...,승용차,남,33세,상해없음,화물차,남,51세,경상,화,경상
3,20190101,01시,0,0,1,0,중구,회현동2가,차대차,측면충돌,...,승용차,남,58세,상해없음,승용차,남,62세,경상,화,경상
4,20190101,01시,0,0,1,0,성동구,행당동,차대사람,횡단중,...,승용차,남,30세,상해없음,보행자,남,32세,경상,화,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39253,20191231,22시,0,0,0,1,중랑구,면목동,차대사람,차도통행중,...,화물차,남,28세,상해없음,보행자,여,33세,부상신고,화,부상신고
39254,20191231,22시,0,0,1,0,강남구,삼성동,차대차,기타,...,승용차,여,41세,상해없음,승용차,여,61세,경상,화,경상
39255,20191231,22시,0,0,1,0,송파구,송파동,차대사람,횡단중,...,승용차,남,36세,상해없음,보행자,여,18세,경상,화,경상
39256,20191231,23시,0,0,0,2,강남구,역삼동,차량단독,기타,...,승용차,남,59세,상해없음,없음,없음,없음,없음,화,부상신고


In [7]:
# DataFrame 통합
Seoul0719 = pd.concat([Seoul_0709, Seoul_1018, Seoul_Gyeonggi_1419], ignore_index=True)

# 발생일 데이터타입 str로 변환
Seoul0719['발생일'] = Seoul0719['발생일'].astype(str)

# 중간결과 출력
Seoul0719

,발생일,발생시간,요일,발생지_시군구,법정동명,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,도로형태_대분류,도로형태,가해자차종,가해자성별,가해자연령,가해자신체상해정도,피해자차종,피해자성별,피해자연령,피해자신체상해정도
0,20070101,00시,월,금천구,시흥동,중상,0,1,0,0,...,교차로,교차로내,승용차,남,24세,상해없음,화물차,남,23세,중상
1,20070101,01시,월,성북구,종암동,부상신고,0,0,0,1,...,단일로,교량위,승용차,남,43세,기타불명,없음,없음,불명,없음
2,20070101,01시,월,동대문구,청량리동,중상,0,1,0,0,...,교차로,교차로내,승용차,남,29세,상해없음,보행자,남,44세,중상
3,20070101,01시,월,마포구,서교동,중상,0,1,0,0,...,단일로,기타단일로,원동기장치자전거,남,24세,중상,화물차,남,25세,상해없음
4,20070101,01시,월,강서구,화곡동,중상,0,1,0,0,...,단일로,기타단일로,승용차,여,45세,상해없음,이륜차,남,19세,중상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526982,20191231,22시,화,중랑구,면목동,부상신고,0,0,0,1,...,단일로,기타단일로,화물차,남,28세,상해없음,보행자,여,33세,부상신고
526983,20191231,22시,화,강남구,삼성동,경상,0,0,1,0,...,단일로,기타단일로,승용차,여,41세,상해없음,승용차,여,61세,경상
526984,20191231,22시,화,송파구,송파동,경상,0,0,1,0,...,교차로,교차로부근,승용차,남,36세,상해없음,보행자,여,18세,경상
526985,20191231,23시,화,강남구,역삼동,부상신고,0,0,0,2,...,단일로,기타단일로,승용차,남,59세,상해없음,없음,없음,없음,없음


In [8]:
# 피/가해자 연령에서 숫자를 추출
Seoul0719['가해자연령'] = Seoul0719['가해자연령'].str.extract('(\d+)')
Seoul0719['피해자연령'] = Seoul0719['피해자연령'].str.extract('(\d+)')

# 연령 결측치 레코드 삭제 및 데이터타입 int형으로 변환
Seoul0719 = Seoul0719.dropna(subset=['가해자연령']+['피해자연령'])
Seoul0719['가해자연령'] = Seoul0719['가해자연령'].astype(int)
Seoul0719['피해자연령'] = Seoul0719['피해자연령'].astype(int)

# 피/가해자 성별에서 '기타불명' 레코드 삭제
unknown_sex = Seoul0719[(Seoul0719['가해자성별'] == '기타불명') | (Seoul0719['피해자성별'] == '기타불명')].index
Seoul0719 = Seoul0719.drop(unknown_sex, inplace=False)

Seoul0719.reset_index(inplace=True, drop=True) # index 재정렬

In [9]:
# 통합된 DataFrame에 '주야' column 추가
day_night_condition = [
    ((Seoul0719['발생시간'].str.slice(start=0, stop=2).astype(int) >= 6) & (Seoul0719['발생시간'].str.slice(start=0, stop=2).astype(int) <= 17)),
    ((Seoul0719['발생시간'].str.slice(start=0, stop=2).astype(int) >= 18) & (Seoul0719['발생시간'].str.slice(start=0, stop=2).astype(int) <= 24)),
    ((Seoul0719['발생시간'].str.slice(start=0, stop=2).astype(int) >= 0) & (Seoul0719['발생시간'].str.slice(start=0, stop=2).astype(int) <= 5))]
day_night_list = ['주', '야', '야']
Seoul0719['주야'] = np.select(day_night_condition, day_night_list)

# 중간결과 출력
Seoul0719

,발생일,발생시간,요일,발생지_시군구,법정동명,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,도로형태,가해자차종,가해자성별,가해자연령,가해자신체상해정도,피해자차종,피해자성별,피해자연령,피해자신체상해정도,주야
0,20070101,00시,월,금천구,시흥동,중상,0,1,0,0,...,교차로내,승용차,남,24,상해없음,화물차,남,23,중상,야
1,20070101,01시,월,동대문구,청량리동,중상,0,1,0,0,...,교차로내,승용차,남,29,상해없음,보행자,남,44,중상,야
2,20070101,01시,월,마포구,서교동,중상,0,1,0,0,...,기타단일로,원동기장치자전거,남,24,중상,화물차,남,25,상해없음,야
3,20070101,01시,월,강서구,화곡동,중상,0,1,0,0,...,기타단일로,승용차,여,45,상해없음,이륜차,남,19,중상,야
4,20070101,01시,월,송파구,가락동,경상,0,0,1,0,...,교차로부근,승합차,남,54,상해없음,승용차,여,43,경상,야
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492932,20191231,22시,화,동작구,상도동,중상,0,1,0,0,...,교차로내,승용차,남,60,상해없음,이륜차,남,54,중상,야
492933,20191231,22시,화,중랑구,면목동,부상신고,0,0,0,1,...,기타단일로,화물차,남,28,상해없음,보행자,여,33,부상신고,야
492934,20191231,22시,화,강남구,삼성동,경상,0,0,1,0,...,기타단일로,승용차,여,41,상해없음,승용차,여,61,경상,야
492935,20191231,22시,화,송파구,송파동,경상,0,0,1,0,...,교차로부근,승용차,남,36,상해없음,보행자,여,18,경상,야


In [10]:
# DataFrame의 Column 순서 변경
Seoul0719 = Seoul0719[['발생일', '발생시간', '요일', '주야', '발생지_시군구', '법정동명', '사고내용', '사망자수', '중상자수', '경상자수',
       '부상신고자수', '사고유형_대분류', '사고유형_중분류', '사고유형', '가해자법규위반', '노면상태_대분류', '노면상태',
       '기상상태', '도로형태_대분류', '도로형태', '가해자차종', '가해자성별', '가해자연령', '가해자신체상해정도',
       '피해자차종', '피해자성별', '피해자연령', '피해자신체상해정도']]

In [11]:
# 주야 -> 야 : 0, 주 : 1
day_night = {'야' : 0, '주' : 1} 
Seoul0719['주야코드'] = Seoul0719['주야'].map(day_night)

# 사고내용 -> 사망 : 0, 중상 : 1, 경상 : 2, 부상신고 : 3
case = {'사망' : 0, '중상' : 1, '경상' : 2, '부상신고' : 3} 
Seoul0719['사고내용코드'] = Seoul0719['사고내용'].map(case)

# 차종
# 승용차 : 0, 화물차 : 1, 이륜차 : 2, 승합차 : 3, 자전거 : 4, 원동기장치자전거 : 5, 건설기계 :6, 특수차 : 7, 개인형이동수단(PM) : 8
# 기타 : 9, 불명 : 10, 농기계 : 11, 사륜오토바이(ATV) : 12
vehicle_type = {'승용차' : 0, '화물차' : 1, '이륜차' : 2, '승합차' : 3, '자전거' : 4, '원동기장치자전거' : 5,
                '건설기계' :6, '특수차' : 7, '개인형이동수단(PM)' : 8, '기타' : 9, '불명' : 10, '농기계' : 11,
                '사륜오토바이(ATV)' : 12} 
Seoul0719['가해자차종코드'] = Seoul0719['가해자차종'].map(vehicle_type)

# 기상상태
weather_condition = {'맑음' : 0, '흐림' : 1, '비' : 2, '눈' : 3, '안개' : 4, '기타/불명' : 5}
Seoul0719['기상상태코드'] = Seoul0719['기상상태'].map(weather_condition)
    
# 노면상태
road_condition = {'건조' : 0, '젖음/습기' : 2, '서리/결빙' : 3, '적설' : 4, '해빙' : 5, '침수' : 6, '기타' : 7}
Seoul0719['노면상태코드'] = Seoul0719['노면상태'].map(road_condition)

# 사망여부(사망: 1, 그 외: 0)
survived = {'사망':1, '중상':0, '경상':0, '부상신고':0}
Seoul0719['사망여부'] = Seoul0719['사고내용'].map(survived)

# 피해자연령대
age_condition = [
    (Seoul0719['피해자연령'] < 10),
    ((Seoul0719['피해자연령'] >= 10) & (Seoul0719['피해자연령'] < 20)),
    ((Seoul0719['피해자연령'] >= 20) & (Seoul0719['피해자연령'] < 30)),
    ((Seoul0719['피해자연령'] >= 30) & (Seoul0719['피해자연령'] < 40)),
    ((Seoul0719['피해자연령'] >= 40) & (Seoul0719['피해자연령'] < 50)),
    ((Seoul0719['피해자연령'] >= 50) & (Seoul0719['피해자연령'] < 60)),
    ((Seoul0719['피해자연령'] >= 60) & (Seoul0719['피해자연령'] < 70)),
    ((Seoul0719['피해자연령'] >= 70) & (Seoul0719['피해자연령'] < 80)),
    ((Seoul0719['피해자연령'] >= 80) & (Seoul0719['피해자연령'] < 90)),
    ((Seoul0719['피해자연령'] >= 90) & (Seoul0719['피해자연령'] < 100)),
    (Seoul0719['피해자연령'] >= 100)]
caselist = ['10세 미만', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대', '90대', '100세 이상']
Seoul0719['피해자연령대'] = np.select(age_condition, caselist)

In [12]:
# 발생일로부터 발생연도와 발생월을 int형으로 추출
Seoul0719['발생일'] = Seoul0719['발생일'].astype(str)
Seoul0719['발생연도'] = Seoul0719['발생일'].str.slice(start=0, stop=4).astype(int)
Seoul0719['발생월'] = Seoul0719['발생일'].str.slice(start=4, stop=6).astype(int)

In [13]:
# 2007~2019년에 발생한 서울시 개별사고정보를 csv로 저장
Seoul0719.to_csv('..\data\output\서울시 개별사고정보(2007_2019).csv', encoding='cp949', index=False)

In [14]:
Seoul0719.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492937 entries, 0 to 492936
Data columns (total 37 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   발생일        492937 non-null  object
 1   발생시간       492937 non-null  object
 2   요일         492937 non-null  object
 3   주야         492937 non-null  object
 4   발생지_시군구    492937 non-null  object
 5   법정동명       492849 non-null  object
 6   사고내용       492937 non-null  object
 7   사망자수       492937 non-null  int64 
 8   중상자수       492937 non-null  int64 
 9   경상자수       492937 non-null  int64 
 10  부상신고자수     492937 non-null  int64 
 11  사고유형_대분류   492937 non-null  object
 12  사고유형_중분류   492937 non-null  object
 13  사고유형       492937 non-null  object
 14  가해자법규위반    492936 non-null  object
 15  노면상태_대분류   492937 non-null  object
 16  노면상태       492937 non-null  object
 17  기상상태       492937 non-null  object
 18  도로형태_대분류   492937 non-null  object
 19  도로형태       492937 non-null  object
 20  가해자차

### 3) 서울시 인구 관련 데이터 전처리

In [15]:
s_pop = pd.read_csv('..\data\input\Seoul_population.txt', sep = "\t", encoding="utf-8")

# 불필요한 index 제거
s_pop.drop([0, 1], inplace=True)
s_pop.reset_index(inplace=True, drop=True)

drop_idx = s_pop[s_pop['자치구'] == '합계'].index
s_pop.drop(drop_idx, inplace=True)
s_pop.reset_index(inplace=True, drop=True)

# 불필요한 column 제거
s_pop.drop(['세대', '인구밀도', '인구밀도.1', '인구.1', '인구.2', '인구.3', '인구.4', '인구.5', '인구.6', '인구.7', '인구.8', '세대당인구', '65세이상고령자'],
           axis=1, inplace=True)

# column명 변경
s_pop.rename(columns={'기간':'연도'}, inplace=True)

# 인구 Column의 data형 변환
s_pop['인구'] = s_pop['인구'].str.replace(',', '').astype(int)

# 연도, 자치구 순서로 오름차순 정렬
s_pop.sort_values(by=['연도', '자치구'], inplace=True)

# index 재정렬
s_pop.reset_index(inplace=True, drop=True)

In [16]:
# 새로운 DataFrame 생성 전, 발생 건수를 list로 추출
case_count = []
year_local_count = pd.DataFrame(Seoul0719.groupby(['발생연도', '발생지_시군구']).size(), columns=['발생건수'])

for i in range(13):
    for j in range(25):
        case_count.append(int(year_local_count['발생건수'][2007+i][j]))

In [17]:
# 발생건수 DataFrame 생성
ser_dict = {'발생건수':case_count}
df_tmp = pd.DataFrame(ser_dict)

# DataFrame 병합
s_pop = s_pop.join(df_tmp)

# 인구비율에 따른 발생건수를 계산한 Column 추가
s_pop['인구 수 대비 사고비율'] = s_pop['발생건수'] / s_pop['인구'] * 100

# csv로 저장
s_pop.to_csv('..\data\output\연도별 서울 자치구 인구수와 사건발생 건수.csv', encoding='cp949', index=False)